In [ ]:
#import important stuff
from gpaw import GPAW, PW, FermiDirac
from ase.io import read
from ase.spectrum.band_structure import BandStructurePlot
from ase.build import mx2
import numpy as np
import matplotlib.pyplot as plt

#structure parameters
structure_params = {'MoS2': {'kind': '2H', 'a': 3.184, 'thickness': 3.127},
                       'MoSe2': {'kind': '2H', 'a': 3.320, 'thickness': 3.338},
                       'WS2': {'kind': '2H', 'a': 3.186, 'thickness': 3.359},
                       'WSe2': {'kind': '2H', 'a': 3.319, 'thickness': 3.146},
                       'BN': {'kind': '2H', 'a': 2.510, 'thickness': 1}}

formula_List=['MoS2','MoSe2','WS2','WSe2']
sym_path_List=['KM','KG']
nP_List=[30,59]
for i in range(len(formula_List)):
    for j in range(2):
        #choose
        formula = formula_List[i]
        vac = 20
        out_dir = './out/'
        name = out_dir + formula + '_PBE_gs.gpw'
        sym_path=sym_path_List[j]
        nP=nP_List[j]

        #build structure
        s = structure_params[formula]
        structure = mx2(formula=formula, kind=s['kind'], a=s['a'], thickness=s['thickness'],
                    size=(1, 1, 1), vacuum=vac)
        #choose path in kspace and load converged density
        kpts = structure.cell.bandpath(path=sym_path, npoints=nP,
                                   pbc=structure.pbc, eps=0.1)#what was eps??
        calc = GPAW(name)
        #calculate bandstructure
        emptybands=13
        convbands = emptybands // 2
        parms = {
            'basis': 'dzp',
            'nbands': -emptybands,
            'txt': 'bs.txt',
            'fixdensity': True,
            'kpts': kpts,
            'convergence': {
                'bands': -convbands},
            'symmetry': 'off'}
        calc = GPAW(name, **parms)
        calc.get_potential_energy()
        #save bandstructure
        bs = calc.band_structure()
        bs.write('bsLoop_formula_'+formula+'_direction_'+sym_path+'_npoints_{}_emptybands_{}'.format(nP,emptybands)+'.json')

In [4]:
#import important stuff
from gpaw import GPAW, PW, FermiDirac
from ase.io import read
from ase.spectrum.band_structure import BandStructurePlot
from ase.spectrum.band_structure import BandStructure
from ase.build import mx2
import numpy as np
import matplotlib.pyplot as plt
import ase.units
Hartree = ase.units.Hartree#Hartree energy
Bohr = ase.units.Bohr#Bohr radius
#
formula = 'MoS2'
sym_path='KM'
nP=30
emptybands=13
bs1=BandStructure.read('bsLoop_formula_'+formula+'_direction_'+sym_path+'_npoints_{}_emptybands_{}'.format(nP,emptybands)+'.json')
#
Nkpt=np.shape(bs1.energies)[1]
kPath=bs1.path.kpts
kDiff=kPath-kPath[0]
DeltaK_mag=np.zeros((Nkpt))
for i in range(Nkpt):
    DeltaK_mag[i]=np.sqrt((kPath[i,0]-kPath[0,0])**2+(kPath[i,1]-kPath[0,1])**2+(kPath[i,2]-kPath[0,2])**2)
#
energybands=bs1.energies[0]
valenceBand=energybands[:,12]
conductionBand=energybands[:,13]
#choose a cut off
k_cut=0.015
bool_k=DeltaK_mag<k_cut
DeltaK_cut=DeltaK_mag[bool_k]
valenceBand_cut=valenceBand[bool_k]
conductionBand_cut=conductionBand[bool_k]
#
kVec=np.concatenate((np.sort(-DeltaK_cut),DeltaK_cut[1:]))
valVec=np.concatenate((np.sort(valenceBand_cut),valenceBand_cut[1:]))
conVec=np.concatenate((-np.sort(-conductionBand_cut),conductionBand_cut[1:]))
print(kVec)
print(valVec)
print(conVec)
#
#fit
from scipy.optimize import curve_fit
xData=kVec
y1Data=valVec
y2Data=conVec
#fit function
atomUnit_to_eV=Hartree
Aang_to_atomUnit=1/Bohr
#apparently k is already in atomic units!!!
def val_band(x,A1,m_eff_h):
    return A1-atomUnit_to_eV*(x)**2/(2*m_eff_h)
def con_band(x,A1,m_eff_e):
    return A1+atomUnit_to_eV*(x)**2/(2*m_eff_e)
parameters1, _ = curve_fit(val_band, xData, y1Data)
parameters2, _ = curve_fit(con_band, xData, y2Data)
print(parameters1)
print(parameters2)
fit_val=val_band(xData,parameters1[0],parameters1[1])
fit_con=con_band(xData,parameters2[0],parameters2[1])
print(fit_val)
#print(valVec)
print(fit_con)

[-0.00812766 -0.          0.00812766]
[-4.06419507 -4.06083408 -4.06419507]
[-2.39496019 -2.39920275 -2.39496019]
[-4.06307478e+00 -1.70694179e+04]
[-2.39637436e+00 -6.65117093e+04]
[-4.06307473 -4.06307478 -4.06307473]
[-2.39637438 -2.39637436 -2.39637438]
